In [1]:
script = False

import sys
import pandas as pd
import smatch

if(script): 
    from tqdm import tqdm
else:
    from tqdm.notebook import tqdm

In [2]:
hpix = '16384'

data_karina = pd.read_csv('karina/karina_' + hpix + '.csv')
data_jacobs = pd.read_csv('jacobs/jacobs_' + hpix + '.csv')
data_jimena = pd.read_csv('jimena/jimena_' + hpix + '.csv')

data_karina = data_karina.drop(['HPIX_128', 'HPIX_64'], axis=1)
data_jacobs = data_jacobs.drop(['HPIX_128', 'HPIX_64'], axis=1)
data_jimena = data_jimena.drop(['HPIX_128', 'HPIX_64'], axis=1)

data_karina.rename({'RA': 'K_RA', 'DEC': 'K_DEC', 'SCORE': 'K_SCORE'}, axis=1, inplace=True)
data_jacobs.rename({'RA': 'J_RA', 'DEC': 'J_DEC'}, axis=1, inplace=True)
data_jimena.rename({'RA': 'G_RA', 'DEC': 'G_DEC'}, axis=1, inplace=True)

In [3]:
#Cross-matching Jacobs & Karina
ra1, dec1 = data_karina['K_RA'], data_karina['K_DEC']
ra2, dec2 = data_jacobs['J_RA'], data_jacobs['J_DEC']
matches = smatch.match(ra1, dec1, 0.00028, ra2, dec2, nside=32, maxmatch=1) #0.0024

data_JK = pd.DataFrame()
for i in tqdm(range(len(matches))):
    point = pd.concat( (data_karina.iloc[matches[i][0]], data_jacobs.iloc[matches[i][1]]), axis=0)
    data_JK = data_JK.append(point, ignore_index=True)

  0%|          | 0/85 [00:00<?, ?it/s]

In [4]:
#Cross-matching with me, using Karina's coordinates
ra1, dec1 = data_JK['K_RA'], data_JK['K_DEC']
ra2, dec2 = data_jimena['G_RA'], data_jimena['G_DEC']
matches = smatch.match(ra1, dec1, 0.00028, ra2, dec2, nside=32, maxmatch=1) #0.0024

data_JKG = pd.DataFrame()
for i in tqdm(range(len(matches))):
    point = pd.concat( (data_JK.iloc[matches[i][0]], data_jimena.iloc[matches[i][1]]), axis=0)
    data_JKG = data_JKG.append(point, ignore_index=True)
    
#Cross-matching with me, using Jacobs's coordinates
ra1, dec1 = data_JK['J_RA'], data_JK['J_DEC']
ra2, dec2 = data_jimena['G_RA'], data_jimena['G_DEC']
matches = smatch.match(ra1, dec1, 0.00028, ra2, dec2, nside=32, maxmatch=1) #0.0024

for i in tqdm(range(len(matches))):
    point_tmp = data_JK.iloc[matches[i][0]]
    if(point_tmp['Y3_COADD_OBJECT_ID'] in data_JKG['Y3_COADD_OBJECT_ID'].values):
        continue
    point = pd.concat( (data_JK.iloc[matches[i][0]], data_jimena.iloc[matches[i][1]]), axis=0)
    data_JKG = data_JKG.append(point, ignore_index=True)
    
data_JKG = data_JKG.drop_duplicates()
data_JKG = data_JKG.astype({'COADD_OBJECT_ID': int, 'Y3_COADD_OBJECT_ID': int})
data_JKG.to_csv('JKG_'+ str(hpix) + '.csv', index=False)

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

In [5]:
data_JKG

,K_RA,K_DEC,K_SCORE,Y3_COADD_OBJECT_ID,J_RA,J_DEC,score_sims,score_real,score_both,COADD_OBJECT_ID,...,G_DEC,SINGLE,RING,SMOOTH,COMPANIONS,SDSS_SPIRALS,DES_SPIRALS,CROWDED,ARTIFACTS,MOST_NEGATIVES
0,0.042760,-41.586881,0.000012,177109235,0.042760,-41.586881,0.0000,0.0,0.0,1031403632,...,-41.586922,0.0014,0.0006,0.2133,0.0022,0.0007,0.0008,0.0005,0.0026,0.7779
1,0.230468,-41.505303,0.000007,178763327,0.230468,-41.505303,0.0000,0.0,0.0,1035165930,...,-41.505347,0.0000,0.0000,0.0000,0.0000,0.0002,0.0001,0.0000,0.0001,0.9995
2,0.080925,-41.407771,0.000009,177100029,0.080925,-41.407771,0.0000,0.0,0.0,1031387673,...,-41.407818,0.0000,0.0000,0.0002,0.0001,0.0015,0.0016,0.0001,0.0009,0.9955
3,0.006490,-41.528511,0.000010,177106095,0.006490,-41.528511,0.0000,0.0,0.0,1031398234,...,-41.528546,0.0001,0.0002,0.0002,0.0007,0.0027,0.0041,0.0002,0.0034,0.9883
4,0.003243,-41.457142,0.000015,177102448,0.003243,-41.457142,0.0002,0.0,0.0,1031391823,...,-41.457186,0.0004,0.9886,0.0050,0.0000,0.0002,0.0002,0.0000,0.0002,0.0053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,359.561904,-40.731106,0.000005,175493363,359.561904,-40.731106,0.0001,0.0,0.0,1031039089,...,-40.731143,0.0005,0.0003,0.0016,0.0790,0.0164,0.0009,0.0007,0.0019,0.8987
81,359.981490,-40.801917,0.000012,175497108,359.981490,-40.801917,0.1110,0.0,0.0,1031045328,...,-40.801957,0.0001,0.0001,0.0006,0.0001,0.0003,0.0001,0.0000,0.0001,0.9986
82,359.873174,-40.728198,0.000029,175493137,359.873174,-40.728198,0.0000,0.0,0.0,1031038304,...,-40.728227,0.0001,0.0002,0.0001,0.0000,0.0012,0.0003,0.0000,0.0001,0.9979
83,359.900499,-40.610033,0.000254,175487730,359.900499,-40.610033,0.9969,0.0,0.0,1031028800,...,-40.610066,0.0062,0.0046,0.9269,0.0016,0.0005,0.0004,0.0002,0.0012,0.0583
